In [1]:
import csv
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
from typing import Optional, Tuple
import folium

In [22]:
# Carica il dataset
dataSet = pd.read_csv('validated_delivery_data_ml_imputed.csv')

dataSet.head()
datasetTest=dataSet#.head(20)
dataSet.head()

,Unnamed: 0,SERVICETYPE,VEHICLETYPE,DEPARTURE_COUNTRY,DEPARTURE_ZIPCODE,ARRIVAL_COUNTRY,ARRIVAL_ZIPCODE,SHIPPING_DATE,GROSS_WEIGHT_KG,NET_WEIGHT_KG,VOLUME_M3,DECLARED_DISTANCE_KM,ACTUAL_DELIVERY_DATE,DELIVERY_TIME_HH,WDAY
0,1,Via gomma,Standard,IT,62010.0,IT,20121,2022-11-16 00:00:14,7.30,7.00,0.0,348.76,2023-01-11 00:00:14,1344,2
1,2,E-commerce Resi,Standard,IT,62010.0,IT,15069,2022-11-16 00:00:14,1.70,1.40,0.0,348.76,2023-01-11 00:00:14,1344,2
2,3,Via gomma,Standard,IT,62010.0,IT,10036,2022-11-17 00:00:14,0.72,0.42,0.0,348.76,2023-01-11 00:00:14,1320,2
3,4,E-commerce Resi,Standard,IT,62010.0,IT,70121,2022-11-16 00:00:14,0.86,0.56,0.0,348.76,2023-01-11 00:00:14,1344,2
4,5,Via gomma,Standard,IT,62010.0,IT,20090,2022-11-16 00:00:14,0.58,0.28,0.0,348.76,2023-01-11 00:00:14,1344,2


In [ ]:
def get_zipcode_coordinates(zipcode: str, country) -> Optional[Tuple[float, float]]:
    try:
        geolocator = Nominatim(user_agent="my_distance_calculator")
        location = geolocator.geocode(f"{zipcode}, {country}")
       
        if location:
            return (location.latitude, location.longitude)
        return None
       
    except Exception as e:
        print(f"Error getting coordinates for {zipcode}: {str(e)}")
        return None

        
viaggi=[]#inserisco le cordinate della partenza, coordinarte dell'arrivo
citta={} #inserisce il cap con le sue coordinate

for _, row in datasetTest.iterrows():
    partenza = get_zipcode_coordinates(row["DEPARTURE_ZIPCODE"], row["DEPARTURE_COUNTRY"])
    arrivo = get_zipcode_coordinates(row["ARRIVAL_ZIPCODE"], row["ARRIVAL_COUNTRY"])
    if arrivo is not None and partenza is not None:
        tratta = [partenza, arrivo]
        viaggi.append(tratta)
        citta[str(row["DEPARTURE_ZIPCODE"])] = partenza
        citta[str(row["ARRIVAL_ZIPCODE"])] = arrivo

#print(viaggi)


Error getting coordinates for 95030.0: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=95030.0%2C+IT&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))
Error getting coordinates for 63076.0: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=63076.0%2C+IT&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))
Error getting coordinates for 187.0: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=187.0%2C+IT&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))


In [18]:
# Creare la mappa
mappa = folium.Map(location=[45, 10], zoom_start=10)

# Aggiungere percorsi
for viaggio in viaggi:
    folium.PolyLine(viaggio, color="blue", weight=2, opacity=0.7).add_to(mappa)

for nome, coord in citta.items():
    folium.Marker(coord, popup=nome, icon=folium.Icon(color="red")).add_to(mappa)

mappa.save("mappa_viaggi.html")
print("Mappa aggiornata con più viaggi salvata come 'mappa_viaggi.html'")


Mappa aggiornata con più viaggi salvata come 'mappa_viaggi.html'


In [ ]:
#funzione che calcola le coordinate, passandoglio il cap e la nazionalità, re
def get_zipcode_coordinates(zipcode: str, country) -> Optional[Tuple[float, float]]:
    try:
        geolocator = Nominatim(user_agent="my_distance_calculator")
        location = geolocator.geocode(f"{zipcode}, {country}")
       
        if location:
            return (location.latitude, location.longitude)
        return None
       
    except Exception as e:
        print(f"Error getting coordinates for {zipcode}: {str(e)}")
        return None

#funzione che disegna su una mappa.html le tratte dei viaggi. Prende come parametro il nome del file.csv e il nome con cui salvare il file.html(scrivere senza .html)
def disegnaMappa(nomeFilecsv,nomeMappa):
    dataSet = pd.read_csv(nomeFilecsv) #legge il file

    #se non riesce a caricare tutte le tratte, si inidca solo determnate tratte da disegnare
    datasetTest=dataSet#.head(20)

    viaggi=[]#inserisco le cordinate della partenza, coordinarte dell'arrivo
    citta={} #inserisce il cap con le sue coordinate

    #cicla il dataset per calcolare per ogni cap, di partenza e arrivo, le proprie coordinate
    for _, row in datasetTest.iterrows():
        partenza = get_zipcode_coordinates(row["DEPARTURE_ZIPCODE"], row["DEPARTURE_COUNTRY"])
        arrivo = get_zipcode_coordinates(row["ARRIVAL_ZIPCODE"], row["ARRIVAL_COUNTRY"])
        
        #controlla che le coordinate vengano trovate
        if arrivo is not None and partenza is not None:
            #crea tratta e la aggiunge alla lista dei viaggi
            tratta = [partenza, arrivo]
            viaggi.append(tratta)

            #aggiunge le coordinate e il cap nella lista delle città
            citta[str(row["DEPARTURE_ZIPCODE"])] = partenza
            citta[str(row["ARRIVAL_ZIPCODE"])] = arrivo

    # Creare la mappa
    mappa = folium.Map(location=[45, 10], zoom_start=10)
    
    # Aggiungere percorsi
    for viaggio in viaggi:
        folium.PolyLine(viaggio, color="blue", weight=2, opacity=0.7).add_to(mappa)

    #aggiunge il punti a inizio e fine tratta, indicando pure i loro cap corrispondenti
    for nome, coord in citta.items():
        folium.Marker(coord, popup=nome, icon=folium.Icon(color="red")).add_to(mappa)

    #salva file.html
    mappa.save(f"{nomeMappa}.html")
    print(f"Mappa aggiornata con più viaggi salvata come '{nomeMappa}.html'")
    
